<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/250722_NLP_PDF_%EC%9A%94%EC%95%BD%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. finBERT모델 실험

✅ 전체 흐름 개요
- 목표:
지정된 폴더 내 PDF 사업보고서들을 FinBERT 기반 문장 감성 분석 및 요약

- 폴더 경로:
/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서

- 사용 모델:
FinBERT (감성 분석) + 추가로 Huggingface summarization 모델 (bart, t5 등)



## Step 1: Google Drive 마운트

In [23]:

from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

## Step 2: PDF 폴더 경로 지정 및 파일 리스트 확보

In [2]:
import os

# 정확한 폴더 경로
pdf_dir = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"

# 폴더 존재 확인
if not os.path.exists(pdf_dir):
    raise FileNotFoundError(f"폴더를 찾을 수 없습니다: {pdf_dir}")

# PDF 파일 목록 수집
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

# 리스트 확인
print(f"✅ 총 PDF 파일 수: {len(pdf_files)}")
for f in pdf_files[:5]:
    print("📄", f)


✅ 총 PDF 파일 수: 3005
📄 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/대주이엔티 사업보고서제출기한연장신고서 (2024.12).pdf
📄 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/더테크놀로지 사업보고서제출기한연장신고서 (2024.12).pdf
📄 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/씨앗 사업보고서제출기한연장신고서 (2024.12).pdf
📄 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/동성제약 사업보고서 (2024.12).pdf
📄 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/광동헬스바이오 사업보고서 (2024.12).pdf


## Step 3: PyMuPDF로 PDF 텍스트 추출

In [3]:
!pip install -q PyMuPDF
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text


## Step 4: FinBERT 모델 로드 및 finbert_pipleline 정의

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# FinBERT 모델 로드
tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# 감성 분석 파이프라인 생성
finbert_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


## Step 5: 감성 분석 함수 정의



In [5]:
from nltk.tokenize import sent_tokenize

def analyze_sentiment(text):
    sentences = sent_tokenize(text)
    results = []
    for sentence in sentences:
        if len(sentence.strip()) >= 30:  # 너무 짧은 문장은 제외
            sentiment = finbert_pipeline(sentence)[0]
            results.append((sentence, sentiment['label'], sentiment['score']))
    return results


## Step 6: 첫 PDF 파일 분석 실행

In [6]:
import nltk

# Punkt 관련 전체 토크나이저 세트 재설치
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('tokenizers/punkt')
nltk.download('tokenizers/punkt/english.pickle')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Error loading tokenizers/punkt: Package 'tokenizers/punkt'
[nltk_data]     not found in index
[nltk_data] Error loading tokenizers/punkt/english.pickle: Package
[nltk_data]     'tokenizers/punkt/english.pickle' not found in index


False

In [7]:

# 예시용: 첫 번째 PDF 파일 선택
sample_pdf = pdf_files[0]

# 텍스트 추출
pdf_text = extract_text_from_pdf(sample_pdf)

# 감성 분석 실행
results = analyze_sentiment(pdf_text)

# 결과 집계
pos = [r for r in results if r[1] == 'positive']
neg = [r for r in results if r[1] == 'negative']
neu = [r for r in results if r[1] == 'neutral']

# 요약 출력
print(f"\n📘 분석 대상: {os.path.basename(sample_pdf)}")
print(f"🟢 긍정: {len(pos)} | 🔴 부정: {len(neg)} | ⚪ 중립: {len(neu)}")

print("\n📌 대표 긍정 문장:")
for s in pos[:3]: print(" -", s[0])

print("\n📌 대표 부정 문장:")
for s in neg[:3]: print(" -", s[0])



📘 분석 대상: 대주이엔티 사업보고서제출기한연장신고서 (2024.12).pdf
🟢 긍정: 0 | 🔴 부정: 0 | ⚪ 중립: 0

📌 대표 긍정 문장:

📌 대표 부정 문장:


In [8]:
!pip install pytesseract pdf2image
from pdf2image import convert_from_path
import pytesseract

# OCR로 이미지에서 텍스트 추출
def extract_text_ocr(pdf_path):
    pages = convert_from_path(pdf_path, dpi=300)
    text = ""
    for page in pages:
        text += pytesseract.image_to_string(page, lang='eng+kor')  # 한글도 가능하게
    return text

In [9]:
 print(pdf_text[:500])

목                 차
사업보고서 제출 기한 연장 신고서.............................................................................................................1
사업보고서 제출 기한 연장 신고.................................................................................................................2
 
사업보고서 제출 기한 연장 신고서
 
 
 
 
                                    제 35 기 
 
2024년 01월 01일
부터
2024년 12월 31일
까지
금융위원회
한국거래소 귀중
2025 년  3 월   21 일
회      사      명 :
대주이엔티 주식회사
대   표    이   사 :
황광수, 박주봉
본  점  소  재  지 :
인천 중구


# 02. PDF 이슈 => BART 요약 모델 사용

PDF라 finBERT 안됨

In [10]:
# STEP 1. 설치
!pip install -q PyMuPDF transformers

# STEP 2. PDF 텍스트 추출 함수
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text


In [11]:
# STEP 3. BART 요약 모델 불러오기
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


In [12]:
# STEP 4. 요약 함수 정의 (긴 텍스트 분할)
def bart_chunked_summarize(text, chunk_size=1000, max_chunks=5):
    from transformers import BartTokenizer, BartForConditionalGeneration

    # 전처리: 줄바꿈, 점선 제거
    clean_text = text.replace('\n', ' ').replace('·', '.').replace('▪', '-').strip()
    sentences = clean_text.split('. ')  # 문장 단위 나누기

    # chunk 자르기
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) < chunk_size:
            chunk += sentence + '. '
        else:
            chunks.append(chunk.strip())
            chunk = sentence + '. '
        if len(chunks) >= max_chunks:
            break
    if chunk and len(chunks) < max_chunks:
        chunks.append(chunk.strip())

    # 각 chunk 요약
    summaries = []
    for i, c in enumerate(chunks):
        inputs = tokenizer([c], max_length=1024, return_tensors='pt', truncation=True)
        summary_ids = model.generate(
            inputs['input_ids'],
            max_length=150,
            min_length=40,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(f"요약 {i+1}: {summary}")

    return '\n\n'.join(summaries)


In [13]:
# STEP 5. 경로 지정하고 실행
# PDF 경로 입력
pdf_path = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/강원랜드 사업보고서 (2024.12).pdf"

# 텍스트 추출
text = extract_text_from_pdf(pdf_path)
print("✅ 텍스트 추출 완료, 길이:", len(text))

# 요약 실행
if len(text.strip()) < 100:
    print("⚠️ PDF 내용이 너무 짧습니다. 요약 생략.")
else:
    summary = bart_chunked_summarize(text)
    print("📘 요약 결과:\n")
    print(summary)


✅ 텍스트 추출 완료, 길이: 334914
📘 요약 결과:

요약 1: 1. I.   회사의 개요............................................................................................................................................................................................................3 1.        “  고   서......................................................................................................................................4 3. “I.”   “ ” “’”“I’m sorry, I’ve got to go.’ “ “ I. ” I. ‘ ’     “What’s wrong with this picture?”

요약 2: 1.     “   ‘ 개요’,   “   ””, “’”  ‘’  ”’ “,” ‘, ” ’, ’’. 2.   ”. “. ”,  ‚”,. “  “,  , ‘.’,.  .  ,. ”. .” , “;. ’. ‘;.‚’ ,  .,  ;.

요약 3: 위험    ’관한   ‘’  “”  ’’ ’,  ”’”, ‘,’, “,” ‘;. ”,  ”‘, ”.’,.”. ’.”,.’. ‘.  ’ ‘“’;. ”., “. “;. ”; “:” “,  , ‚”: “., ”,. ”;

요약 4: 연 포괄    “  회계”   “” (“ ”)     “  ”” “ ’’ (”’) (‘’ ”) “ “,” ”, ”,’, ‘”.’.”

요약 5: 금    “�    ” (   ‘’ (  “”) (“’’) ( “ ” “, ”, ’”,   ’, “.” ) (‘”’), ”“,”  ‚’. “ ” (’'’). ”. ” ’ ’.’ ”,. ‘. ’,. ”., ‘, ‘;. ‘,. ’., “;. '”

In [14]:
#STEP 6. 전처리

def clean_report_text(text):
    # 줄바꿈 제거 및 점선 제거
    text = text.replace('\n', ' ')
    text = text.replace('..............................................................................................', '')
    # 기호 정리
    for symbol in ['“', '”', '‘', '’', '•', '▪', '·', '″', "'", '"']:
        text = text.replace(symbol, '')
    # 여러 공백 제거
    import re
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [15]:
#STEP 7. 의미 있는 본문만 추출
def extract_meaningful_section(text, keyword="개요"):
    start = text.find(keyword)
    if start == -1:
        return text[:3000]  # 못 찾으면 앞부분 일부라도 사용
    else:
        return text[start:start+5000]  # 해당 키워드부터 5000자 분량만 사용


In [16]:
raw_text = extract_text_from_pdf(pdf_path)
cleaned = clean_report_text(raw_text)
meaningful = extract_meaningful_section(cleaned)
summary = bart_chunked_summarize(meaningful, chunk_size=1000, max_chunks=3)
print(summary)


요약 1: 1.     ‘   회  개  요  1.2. II.  ‘ ’ ‘’   ’’ 1.3. III. ‘.’ 2. “’” 1.4. IV.

요약 2: 1.      ‘현급  회계  금  한      ‘‘”’   “’” (“”) (’)’ (‘)” ’’ ””)(’, ‘, ”, ’) ”. ” “ (”),  ”,    “, “.”.

요약 3: 기타    ‘‘  ’   ‘’ (‘) (’’) (  ’”) ( ‘,’ ’, ‘), ‘ ’ ‘.’ ) (”’,”, ”‘, “, ’.” ) ( ’), ’,.’ , ‘;’,. ‘,.”,. ’;. ‘,'’.


# 03. GPT API를 활용한 자동화 파이프라인

In [17]:
# ✅ 최신 OpenAI SDK (v1.x) 기준 PDF 다중 요약 자동화 코드
# - GPT-3.5-turbo 사용
# - 사업 보고서의 핵심 내용을 약 10줄 이내로 요약

import os
import fitz  # PyMuPDF
from tqdm import tqdm
import openai

# ✅ OpenAI 클라이언트 설정 (v1.x 문법)
client = openai.OpenAI(api_key="")  # 여기에 본인의 API 키 입력

# ✅ PDF 텍스트 추출 함수
def extract_text(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# ✅ 텍스트 전처리 (줄바꿈 제거, 기호 정리 등)
def clean_text(text):
    import re
    text = text.replace('\n', ' ')
    text = re.sub(r'[“”‘’•▪·″]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# ✅ 사업 개요 위주 텍스트 일부만 추출
def extract_section(text, keyword="II. 사업의 내용"):
    start = text.find(keyword)
    return text[start:start+5000] if start != -1 else text[:3000]

# ✅ GPT 요약 함수 (최신 버전)
def gpt_summary(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
    )
    return response.choices[0].message.content

# ✅ PDF 하나 요약 실행 함수
def summarize_file(pdf_path):
    text = extract_text(pdf_path)
    cleaned = clean_text(text)
    section = extract_section(cleaned)
    prompt = f"다음은 사업보고서의 핵심 내용입니다. 약 10줄 이내로 요약해 주세요:\n\n{section}"
    return gpt_summary(prompt)

# ✅ 폴더 내 max_files 개수만큼 PDF 순회 요약 실행
def batch_summarize(input_dir, output_dir, max_files=20):
    os.makedirs(output_dir, exist_ok=True)
    files = [f for f in os.listdir(input_dir) if f.endswith(".pdf")][:max_files]  # ✨ 앞에서 20개만

    for pdf_file in tqdm(files):
        pdf_path = os.path.join(input_dir, pdf_file)
        try:
            summary = summarize_file(pdf_path)
            with open(os.path.join(output_dir, pdf_file.replace(".pdf", ".txt")), "w") as f:
                f.write(summary)
        except Exception as e:
            print(f"❌ 오류 발생: {pdf_file} - {e}")

# ✅ 실행 예시 (경로만 본인 환경에 맞게 수정)
input_dir = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"
output_dir = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/0722_PDF summaries"
batch_summarize(input_dir, output_dir)

  5%|▌         | 1/20 [00:02<00:46,  2.47s/it]

❌ 오류 발생: 대주이엔티 사업보고서제출기한연장신고서 (2024.12).pdf - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


 10%|█         | 2/20 [00:04<00:41,  2.30s/it]

❌ 오류 발생: 더테크놀로지 사업보고서제출기한연장신고서 (2024.12).pdf - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


 15%|█▌        | 3/20 [00:06<00:39,  2.30s/it]

...

❌ 오류 발생: 선샤인푸드 사업보고서 (2024.12).pdf - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


 95%|█████████▌| 19/20 [00:44<00:02,  2.33s/it]

❌ 오류 발생: SBI인베스트먼트 사업보고서 (2024.12).pdf - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


100%|██████████| 20/20 [00:46<00:00,  2.33s/it]

❌ 오류 발생: 유안타제12호스팩 사업보고서 (2024.12).pdf - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


# KoBART Summarization

- 모델명: digit82/kobart-summarization

- 장점:

  - 문서형 긴 텍스트도 요약 가능

  - HuggingFace 기반이므로 로컬 또는 Colab에서도 사용 가능

  - 한국어 학습 최적화됨 (AIHub 뉴스 요약 데이터 기반)

In [18]:
#1. 필요한 패키지 설치

!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install pdfplumber
!pip install transformers sentencepiece


In [20]:
# 2. Google Drive 연동 및 PDF 텍스트 추출

import pdfplumber
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

# PDF 경로
pdf_path = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/강원랜드 사업보고서 (2024.12).pdf"

# 텍스트 추출
all_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        all_text += page.extract_text() + "\n"

print("✅ 텍스트 추출 완료. 전체 길이:", len(all_text))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 텍스트 추출 완료. 전체 길이: 326924


In [21]:
# 3. KoBART 요약 모델 로딩

from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


In [22]:
# 4. 슬라이딩 윈도우 방식 요약 실행

import re

# 전처리: 공백 정리
clean_text = re.sub(r'\s+', ' ', all_text)

# 슬라이딩 윈도우 요약
window_size = 3000
step_size = 2500
summaries = []

for i in range(0, len(clean_text), step_size):
    chunk = clean_text[i:i+window_size]
    input_ids = tokenizer.encode(chunk, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(input_ids, max_length=200, min_length=60, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary_text)
    print(f"🔹 {i//step_size + 1}번째 요약 완료")

# 전체 요약 연결
final_summary = "\n\n".join(summaries)
print("📘 전체 종합 요약 결과:\n")
print(final_summary)
